## Calculate surface-forced watermass transformation
Presently, information about the products, e.g. names of variables is hard-coded. Could improve robustness by specifying generic names.

In [24]:
%load_ext autoreload
%autoreload 2
from dask.diagnostics import ProgressBar
import so_decadal_variability as so
import numpy as np
import xarray as xr
import os
import gsw
from xhistogram.xarray import histogram
import pandas as pd
import glob

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from dask.distributed import Client
client = Client()
client

Client Scheduler: tcp://127.0.0.1:39961 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 56 Memory: 66.80 GB


In [26]:
fluxname='jra55'
oceanname='en4'
onoceangrid=False
density = 'sigma0'
timeslice = slice('1980','2018')
dg = 0.1
density_edges = np.arange(1024,1028.5,dg)
save = True
savesequential = True
# Load data
ds = so.process.loaddata(fluxname,oceanname,timeslice,onoceangrid)
# Get grid
ds, xgrid = so.grid.get_xgcm(ds,gridlon='lon',gridlat='lat')
# Calculate a density variable
ds['sst'] = ds['sst']-273.15
ds['sigma0'] = gsw.sigma0(ds['sa'].isel(depth=0).squeeze(),ds['sst'])
ds['b'] = xr.ones_like(ds['sigma0'])
ds['vol4d'] = ds['cell_area']*ds['dz']
# ds = ds.rename({'longitude':'lon','latitude':'lat','longitude_left':'lon_left','latitude_left':'lat_left'})
ds = ds.isel(depth=0).squeeze().drop(['depth'])
ds = ds.chunk(chunks={'time':12})
# Calculate WMT
G = so.wmt.calc_watermasstransformation(ds,xgrid,density_edges,density=density,b_ones=False,penetrative_sw=False)

# Save
years = pd.to_datetime(G['time'].values).year.unique()
if save:
    if onoceangrid:
        localdir = '../../data/processed/'+fluxname+'_'+oceanname+'_onoceangrid'
    else:
        localdir = '../../data/processed/'+fluxname+'_'+oceanname
        
    if ~os.path.isdir(localdir):
        !mkdir $localdir
        
    if savesequential:
        for year in years:
            temp = G.sel(time=str(year))
            filename = 'G.'+density+'.'+str(dg)+'.'+str(year)
            print(filename)
            temp.to_netcdf(localdir+'/'+filename+'.nc')
    else:
        filename = 'G.'+density+'.'+str(dg)
        G.to_netcdf(localdir+'/'+filename+'.nc')

mkdir: cannot create directory ‘../../data/processed/jra55_en4’: File exists
G.sigma0.0.1.1980
G.sigma0.0.1.1981
G.sigma0.0.1.1982
G.sigma0.0.1.1983
G.sigma0.0.1.1984
G.sigma0.0.1.1985
G.sigma0.0.1.1986
G.sigma0.0.1.1987
G.sigma0.0.1.1988
G.sigma0.0.1.1989
G.sigma0.0.1.1990
G.sigma0.0.1.1991
G.sigma0.0.1.1992
G.sigma0.0.1.1993
G.sigma0.0.1.1994
G.sigma0.0.1.1995
G.sigma0.0.1.1996
G.sigma0.0.1.1997
G.sigma0.0.1.1998
G.sigma0.0.1.1999
G.sigma0.0.1.2000
G.sigma0.0.1.2001
G.sigma0.0.1.2002
G.sigma0.0.1.2003
G.sigma0.0.1.2004
G.sigma0.0.1.2005
G.sigma0.0.1.2006
G.sigma0.0.1.2007
G.sigma0.0.1.2008
G.sigma0.0.1.2009
G.sigma0.0.1.2010
G.sigma0.0.1.2011
G.sigma0.0.1.2012
G.sigma0.0.1.2013
G.sigma0.0.1.2014
G.sigma0.0.1.2015
G.sigma0.0.1.2016
G.sigma0.0.1.2017
G.sigma0.0.1.2018
